In [ ]:
import numpy as np
import pandas as pd
import os
import psycopg2 as psycopg

In [ ]:
# Get the list names of csv files within the directory
def get_csv_list():
    csv_list = []
    for file in os.listdir(os.getcwd()):
        if len(file)>=4 and file[-4:]==".csv":
                csv_list.append(file)
    return csv_list

In [ ]:
# Transform the names so that they become compatable with being SQL table names
def clean_name(name):
    for symbol in clean_name.to_be_replaced_1:
        name = name.replace(symbol,"_")
    for symbol in clean_name.to_be_replaced_2:
        name = name.replace(symbol,"")
    return name

clean_name.to_be_replaced_1 = [" ","_","-",r"/","\\",".",","]
clean_name.to_be_replaced_2 = ["$","%","(",")"]


In [ ]:
# Transform python data type to SQL data and format a string to initilize 
def cols_str(df):
    cols_str.replacements = {
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp'
    }

    return ", ".join( "{} {}".format( col_name, cols_str.replacements[ str(df.dtypes.loc[col_name]) ] )  for col_name in df.columns )

In [ ]:
# AWS RDS info
host = ""
dbname = ""
port = "5432"
user = ""
password = ""

# Establish a connection
conn = psycopg.connect(host=host,database=dbname,user=user,port=port,password=password)
cursor = conn.cursor()
print("connected to database")

# Create a folder to move the uploaded csv file to
os.chdir("/Users/zhongyuanchen/AWS_RDS_Project")
os.system("mkdir Uploaded")

# Get the list of file names
Tables={}
files = get_csv_list()

# SQL statment for copying from csv file
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

# Upload csv file to remote database
for file in files:
    # Read in the csv file as pandas dataframe
    Tables[file]=pd.read_csv(file)
    
    # Change column names to be compatable with being SQL column name
    Tables[file].columns = [ clean_name(name) for name in Tables[file].columns ]
    
    # Crate corresponding table in remote database
    cursor.execute("drop table if exists {};".format(file))
    col_str = cols_str(Tables[file])
    table_name = file[:-4]
    cursor.execute("create table {0} ({1});".format(table_name, col_str))
    
    # open the csv file
    f = open(file)
     
    # Upload csv file to corresponding table in remote database
    cursor.copy_expert(sql=SQL_STATEMENT % table_name, file=f)
    cursor.execute("grant select on table %s to public" % table_name)
    conn.commit()
    print( "Table '{}' has been added to database".format(table_name) )
    f.close()
    
    # Move the csv file to the "Uploaded" folder
    os.system("mv '{}' Uploaded".format(file))
    Tables[file].to_csv(file,index=False)
    
# Close the cursor
cursor.close()
    